In [2]:
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
df=pd.read_excel('dataset_telegram.xlsx')
df.head(5)

,DATE,HEADLINE,TICKER,COMPANY NAME,SECTOR,SUBSECTOR,SERIES,DATE OF LISTING,PAID UP VALUE,MARKET LOT,...,close,volume,adj close,avg High,avg low,avg open,avg close,avg volume,avg adj close,trend
0,2022-02-15,Religare Ent Says\nâ–¶ï¸Religare Finvest vict...,RELIGARE,Religare Enterprises Limited,Finance - Investments,Finance - Investments,EQ,2007-11-21,10,1,...,122.050003,122.050003,910705,159.109998,159.779999,152.860001,156.130000,156.130000,686379.6,bearish
1,2022-02-15,Ashok Leyland targets 40% ICV sales from CNG v...,ASHOKLEY,Ashok Leyland Limited,Auto - LCVs & HCVs,Auto - LCVs & HCVs,EQ,1995-05-25,1,1,...,128.500000,127.602028,15896279,159.329999,161.720001,156.199997,158.689999,157.581058,16145476.0,bearish
2,2022-02-15,Cipla promoters plan to sell 2.5% of stock,CIPLA,Cipla Limited,Pharmaceuticals,Pharmaceuticals,EQ,1995-02-08,2,1,...,921.849976,917.393127,8304774,1135.429993,1155.719995,1129.699988,1145.319995,1139.782751,3662772.4,bullish
3,2022-02-15,"2.05 cr shares of Cipla worth â‚¹1,880 cr trad...",CIPLA,Cipla Limited,Pharmaceuticals,Pharmaceuticals,EQ,1995-02-08,2,1,...,921.849976,917.393127,8304774,1135.429993,1155.719995,1129.699988,1145.319995,1139.782751,3662772.4,bullish
4,2022-02-15,NMDC in focus\n\nIron Ore falls 10% on SGX as ...,NMDC,NMDC Limited,Mining & Minerals,Mining & Minerals,EQ,2008-03-03,1,1,...,149.300003,143.697586,17659797,186.379999,189.089999,182.490005,185.719998,178.750937,18851958.2,bullish


In [4]:
data = pd.concat([df.iloc[:1000, 1:5], df.iloc[:1000, -12:-2]], axis=1)
data['HEADLINE'].replace("[^a-zA-Z0-9\s]"," ",regex=True,inplace=True)

data.head()

,HEADLINE,TICKER,COMPANY NAME,SECTOR,Low,Open,close,volume,adj close,avg High,avg low,avg open,avg close,avg volume
0,Religare Ent Says\n Religare Finvest vict...,RELIGARE,Religare Enterprises Limited,Finance - Investments,125.000000,120.750000,122.050003,122.050003,910705,159.109998,159.779999,152.860001,156.130000,156.130000
1,Ashok Leyland targets 40 ICV sales from CNG v...,ASHOKLEY,Ashok Leyland Limited,Auto - LCVs & HCVs,129.250000,127.199997,128.500000,127.602028,15896279,159.329999,161.720001,156.199997,158.689999,157.581058
2,Cipla promoters plan to sell 2 5 of stock,CIPLA,Cipla Limited,Pharmaceuticals,938.799988,920.099976,921.849976,917.393127,8304774,1135.429993,1155.719995,1129.699988,1145.319995,1139.782751
3,2 05 cr shares of Cipla worth 1 880 cr trad...,CIPLA,Cipla Limited,Pharmaceuticals,938.799988,920.099976,921.849976,917.393127,8304774,1135.429993,1155.719995,1129.699988,1145.319995,1139.782751
4,NMDC in focus\n\nIron Ore falls 10 on SGX as ...,NMDC,NMDC Limited,Mining & Minerals,153.250000,149.050003,149.300003,143.697586,17659797,186.379999,189.089999,182.490005,185.719998,178.750937


# Combine Columns

In [5]:
corpus = []
for row in range(0,len(data.index)):
    corpus.append(' '.join(str(x) for x in data.iloc[row,0:14]))
corpus

['Religare Ent Says\n      Religare Finvest victim of fraud perpetuated by erstwhile mgmt\n      Have challenged process of RFL being classified as    fraud    account \n      RBI on Feb 11 said RFL restructuring can   t be implemented with REL being a promoter as lenders declared RFL a fraud exposure RELIGARE Religare Enterprises Limited Finance - Investments 125.0 120.75 122.0500031 122.0500031 910705 159.1099976 159.7799988 152.8600006 156.1300003 156.1300003',
 'Ashok Leyland targets 40  ICV sales from CNG variants ASHOKLEY Ashok Leyland Limited Auto - LCVs & HCVs 129.25 127.1999969 128.5 127.6020279 15896279 159.3299988 161.7200012 156.1999969 158.6899994 157.5810577',
 'Cipla promoters plan to sell 2 5  of stock CIPLA Cipla Limited Pharmaceuticals 938.7999878 920.0999756 921.8499756 917.3931274 8304774 1135.429993 1155.719995 1129.699988 1145.319995 1139.782751',
 '2 05 cr shares of Cipla worth    1 880 cr trade on NSE at average    915 sh\n\nAlert  Promoters likely to have sold 

In [6]:
y = pd.get_dummies(df['trend']).iloc[:1000,1].values
y

array([False, False,  True,  True,  True, False, False, False,  True,
       False,  True, False, False,  True,  True, False,  True, False,
       False, False, False, False, False,  True,  True, False, False,
       False, False, False, False, False,  True, False, False,  True,
       False, False, False, False, False, False,  True, False,  True,
        True,  True,  True, False,  True, False,  True, False, False,
        True,  True,  True, False, False, False, False, False, False,
       False, False, False,  True, False, False,  True, False, False,
        True, False,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False,

# Featutre Extraction using CountVectorizer

In [7]:
cv = CountVectorizer(ngram_range=(1,2))
X = cv.fit_transform(corpus)
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

# Train Test Split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Training model MultinomialNB

In [9]:
ml = MultinomialNB().fit(X_train, y_train)
y_pred = ml.predict(X_test)

In [10]:
accuracy = accuracy_score(y_test, y_pred)
print("Naive Bayes Accuracy:", accuracy)

confusion_m = confusion_matrix(y_test, y_pred)
print("Naive Bayes Confusion Matrix:\n", confusion_m)

f1 = f1_score(y_test, y_pred)
print("F1 Score:\n", f1)

p=precision_score(y_test,y_pred)
print("Precision Score:\n", f1)

Naive Bayes Accuracy: 0.692
Naive Bayes Confusion Matrix:
 [[82 32]
 [45 91]]
F1 Score:
 0.7027027027027026
Precision Score:
 0.7027027027027026


# Training Model RandomForestClassifier

In [11]:
randomclassifier = RandomForestClassifier(n_estimators=50, criterion='entropy')
rc = randomclassifier.fit(X_train, y_train)
y_pred = rc.predict(X_test)

In [12]:
accuracy = accuracy_score(y_test, y_pred)
print("Random Forest Accuracy:", accuracy)

confusion_m = confusion_matrix(y_test, y_pred)
print("Random Forest Confusion Matrix:\n", confusion_m)

f1 = f1_score(y_test, y_pred)
print("F1 Score:\n", f1)

p=precision_score(y_test,y_pred)
print("Precision Score:\n", f1)

Random Forest Accuracy: 0.668
Random Forest Confusion Matrix:
 [[ 45  69]
 [ 14 122]]
F1 Score:
 0.746177370030581
Precision Score:
 0.746177370030581


# Training model Logistic Regression

In [13]:
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)

In [14]:
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
print("Logistic Regression Accuracy:", accuracy_log_reg)

confusion_m_log_reg = confusion_matrix(y_test, y_pred_log_reg)
print("Logistic Regression Confusion Matrix:\n", confusion_m_log_reg)

f1 = f1_score(y_test, y_pred_log_reg)
print("F1 Score:\n", f1)

p=precision_score(y_test,y_pred_log_reg)
print("Precision Score:\n", f1)

Logistic Regression Accuracy: 0.672
Logistic Regression Confusion Matrix:
 [[ 67  47]
 [ 35 101]]
F1 Score:
 0.7112676056338029
Precision Score:
 0.7112676056338029


# Building RNN Model

In [15]:
model = Sequential([
    Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train.toarray(), y_train, epochs=5, batch_size=64, validation_data=(X_test.toarray(), y_test), callbacks=[EarlyStopping(patience=3)])



Epoch 1/5


12/12 [==============================] - 1s 52ms/step - loss: 0.6462 - accuracy: 0.5987 - val_loss: 0.6099 - val_accuracy: 0.6040
Epoch 2/5
12/12 [==============================] - 0s 39ms/step - loss: 0.4147 - accuracy: 0.9027 - val_loss: 0.5866 - val_accuracy: 0.6360
Epoch 3/5
12/12 [==============================] - 0s 39ms/step - loss: 0.2272 - accuracy: 0.9960 - val_loss: 0.5558 - val_accuracy: 0.6680
Epoch 4/5
12/12 [==============================] - 0s 40ms/step - loss: 0.0920 - accuracy: 1.0000 - val_loss: 0.5259 - val_accuracy: 0.6840
Epoch 5/5
12/12 [==============================] - 0s 35ms/step - loss: 0.0286 - accuracy: 1.0000 - val_loss: 0.5544 - val_accuracy: 0.6880


In [16]:
loss, accuracy = model.evaluate(X_test.toarray(), y_test)
print("RNN Model Accuracy:", accuracy)

y_pred_rnn = model.predict(X_test.toarray())
y_pred_rnn = (y_pred_rnn > 0.5).astype(int)
confusion_m_rnn = confusion_matrix(y_test, y_pred_rnn)
print("RNN Model Confusion Matrix:\n", confusion_m_rnn)

f1 = f1_score(y_test, y_pred_rnn)
print("F1 Score:\n", f1)

p=precision_score(y_test,y_pred_rnn)
print("Precision Score:\n", f1)

8/8 [==============================] - 0s 4ms/step - loss: 0.5544 - accuracy: 0.6880
RNN Model Accuracy: 0.6880000233650208
8/8 [==============================] - 0s 5ms/step
RNN Model Confusion Matrix:
 [[84 30]
 [48 88]]
F1 Score:
 0.6929133858267716
Precision Score:
 0.6929133858267716
